In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

ACADEMIC_YEAR = "113"
SEMESTER = "2"
BASE_URL = "https://course.thu.edu.tw/view/"

def get_data(course_number):
    course_info = { # 使用字典儲存課程資訊
        "課程代碼": course_number,
        "網頁標題": "找不到",
        "授課教師": "找不到",
        "課程類型": "找不到",
        "學分數": "找不到",
        "上課地點": "找不到",
        "課程概述": "找不到" # 新增 "課程概述" 欄位
    }
    try:
        response = requests.get(BASE_URL+ACADEMIC_YEAR+'/'+SEMESTER+'/'+course_number)
        response.raise_for_status()  # 檢查請求是否成功
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.title.string
        course_info["網頁標題"] = title # 儲存網頁標題

        teacher_div = soup.find('div', class_='three columns') # 找到 class="three columns" 的 div
        if teacher_div: # 檢查是否找到 div
            teacher_link = teacher_div.find('a') # 在 div 裡面找到 <a> 標籤
            if teacher_link: # 檢查是否找到 <a> 標籤
                teacher_name = teacher_link.text.strip() # 取得 <a> 標籤的文字並去除空白
                course_info["授課教師"] = teacher_name # 儲存授課教師

        # --- 抓取基本資料 ---
        basic_data_div = soup.find('div', class_='column one-third')
        if basic_data_div:
            basic_data_paragraph = basic_data_div.find('p')
            if basic_data_paragraph:
                basic_data_text = basic_data_paragraph.text.strip()
                data_lines = basic_data_text.split('<br>')

                for line in data_lines:
                    cleaned_line = line.strip().split('\n')
                    if cleaned_line:
                        if cleaned_line[0].startswith("必修課") or cleaned_line[0].startswith("選修課"): # 判斷是否為 必修/選修 資訊
                            course_type = cleaned_line[0].split("，")[0] # 取 "，" 前面的文字 (例如 "必修課")
                            course_info["課程類型"] = course_type # 儲存課程類型
                        if len(cleaned_line[0].split("，")) > 1 and cleaned_line[0].split("，")[1].startswith("學分數"): # 判斷是否為 學分數 資訊 and prevent index error
                            credits_str = cleaned_line[0].split("，")[1]
                            credits = credits_str.split("：")[1].strip() # 取 "：" 後面的文字並去除空白 (例如 "0-2")
                            course_info["學分數"] = credits # 儲存學分數
                        if len(cleaned_line) > 1 and cleaned_line[1].strip().startswith("上課時間"): # 判斷是否為 上課時間/地點 資訊
                            location_info = cleaned_line[1].split("：")[1].strip() # 取 "：" 後面的文字並去除空白 (例如 "三/5,6[芳華廳]")
                            #  進一步處理上課地點 (假設地點在 "[]" 裡面)
                            if "[" in location_info and "]" in location_info:
                                location = location_info.split("[")[1].split("]")[0] # 取 "[" 和 "]" 中間的文字 (例如 "芳華廳")
                            else:
                                location = location_info # 如果沒有 "[]"，則直接使用整段文字
                            course_info["上課地點"] = location # 儲存上課地點

        # --- 抓取課程概述 ---
        description_div = soup.find('div', class_='thirteen columns') # 找到 class="thirteen columns" 的 div
        if description_div: # 檢查是否找到 div
            description_paragraph = description_div.find('p') # 在 div 裡面找到 <p> 標籤
            if description_paragraph: # 檢查是否找到 <p> 標籤
                description_text = description_paragraph.text.strip() # 取得 <p> 標籤的文字並去除空白
                course_info["課程概述"] = description_text # 儲存課程概述


    except requests.exceptions.RequestException as e:
        print("網路請求錯誤:", e)
    except AttributeError:
        print("找不到網頁標題")
    return course_info # 返回課程資訊字典

def save_to_csv(course_data_list, filename="course_data.csv"):
    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile: # 使用 utf-8-sig 編碼，避免中文亂碼
        fieldnames = ["課程代碼", "網頁標題", "授課教師", "課程類型", "學分數", "上課地點", "課程概述"] # CSV 檔案標題列 # 新增 "課程概述" 欄位
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader() # 寫入標題列
        for data in course_data_list:
            writer.writerow(data) # 寫入每一筆課程資料

if __name__ == '__main__': # 確保 save_to_csv 函式定義在最前面或在使用前
    all_course_data = [] # 儲存所有課程資料的列表
    for course in range(3001, 4557): # 爬取的課程代碼範圍
        course_data = get_data(str(course)) # 爬取單一課程資料
        all_course_data.append(course_data) # 將課程資料加入列表

    save_to_csv(all_course_data) # 將所有課程資料寫入 CSV 檔案
    print("課程資料已儲存至 course_data.csv")

In [ ]:
def save_to_csv(course_data_list, filename="course_data.csv"):
    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
        fieldnames = ["課程代碼", "網頁標題", "授課教師", "課程類型", "學分數", "上課地點", "課程概述"]
        writer = csv.DictWriter(
            csvfile,
            fieldnames=fieldnames,
            quoting=csv.QUOTE_MINIMAL,  # 設定 quoting 方式為 QUOTE_MINIMAL
            escapechar='\\',           # 設定 escapechar 為反斜線 (必要時使用)
            doublequote=False          # 設定 doublequote 為 False，避免雙引號被加倍跳脫
        )

        writer.writeheader()
        for data in course_data_list:
            writer.writerow(data)

In [ ]:
    save_to_csv(all_course_data) # 將所有課程資料寫入 CSV 檔案